In [1]:
import os, sys

root = os.path.dirname(os.getcwd())
sys.path.append(root)
print(f"{root = }")
    
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

root = '/home/juliekuo/projects/story_structure'


In [2]:
import mlflow
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

from src.utils import Log, helper_function

In [4]:
# get logger
log = Log()
logger = log.set_logger(file_path = f"{root}/logs/log.log", level = 1, freq = "D", interval = 10, backup = 3, name = "log")

paths = {
    'data': f'{root}/data',
    'reports': f'{root}/reports',
    'models': f'{root}/models',
    'src': f'{root}/src',
}

params = {
    'init': {
            'paths': paths,
        },
}

In [5]:
class Train():
    def __init__(self, paths, logger):
        self.paths = paths
        self.logger = logger
    def setting_mlflow(self):
        """
        setting mlflow configuration to record training process
        """
        mlflow.set_tracking_uri(self.exp_config['mlflow']['tracking_uri'])
        mlflow.set_experiment(self.exp_config['mlflow']['experiment_name'])
        mlflow.start_run(run_name=self.exp_config['mlflow']['run_name'])
        mlflow.set_tags(self.exp_config['mlflow']['tags'])
        os.environ['LOGNAME'] = self.exp_config['mlflow']['log_name']

    def finish_mlflow(self):
        """
        finish mlflow run
        """
        mlflow.end_run()

    def log_to_mlflow(self, model, tokenizer, log_model_flag = False):
        """
        log model to mlflow server
        """
        if log_model_flag:
            # log model to mlflow server
            components = {
                "model": model,
                "tokenizer": tokenizer
                }
            mlflow.transformers.log_model(
                transformers_model=components,
                artifact_path="tagging_model",
            )
        else:
            # log model path
            model_path = f"{self.paths['models']}/{self.exp_config['model']['name']}/{self.exp_config['model']['description']}"
            mlflow.log_param("model_path", model_path)

        # log config file
        mlflow.log_artifact(f"{self.paths['src']}/config/experiment/config.yml")

# class object
self = Train(**params["init"], logger=logger)

In [6]:
self.exp_config = helper_function.load_config(f"{paths['src']}/config/experiment/config.yml")
self.exp_config

{'mlflow': {'tracking_uri': 'http://localhost:5000',
  'experiment_name': 'story-structure',
  'run_name': 'TinyLlama',
  'log_name': 'julie_kuo',
  'device_id': '5',
  'tags': {'base_model': 'TinyLlama',
   'type': 'CAUSAL_LM',
   'loss': 'cross_entropy',
   'dataset': 'percy_jackson'}},
 'data': {'file_path': '/raw/percy_jackson.txt'},
 'model': {'name': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
  'description': 'TinyLlama',
  'checkpoint': 'checkpoint-300',
  'chunk_size': 256,
  'early_stopping_patience': 10,
  'seed': 42},
 'training_args': {'eval_steps': 20,
  'save_steps': 100,
  'logging_steps': 20,
  'max_steps': 20000,
  'save_total_limit': 20,
  'per_device_train_batch_size': 2,
  'per_device_eval_batch_size': 2,
  'gradient_accumulation_steps': 8,
  'gradient_checkpointing': True,
  'gradient_checkpointing_kwargs': {'use_reentrant': False},
  'warmup_steps': 100,
  'learning_rate': 0.0001,
  'optim': 'paged_adamw_8bit',
  'logging_strategy': 'steps',
  'eval_strategy': 'steps',

In [7]:
self.setting_mlflow()

In [ ]:
# set seed and cudnn
helper_function.set_seed(self.exp_config["model"]["seed"])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Fine-tuning model with peft

In [8]:
def load_data(file_path):
    """載入並預處理文本數據"""
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # 將文本分割成較小的段落，使用較短的長度
    self.chunk_size = self.exp_config["model"]["chunk_size"]
    chunks = [text[i:i+self.chunk_size] for i in range(0, len(text), self.chunk_size)]
    
    # 創建dataset
    dataset = Dataset.from_dict({
        'text': chunks
    })
    
    return dataset

In [9]:
def prepare_model_and_tokenizer(model_name):
    """準備小型基礎模型和分詞器，使用新的量化配置"""
    # 設定 4bit 量化配置
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  # 啟用 4bit 載入，以減少模型大小和記憶體使用量
        bnb_4bit_quant_type="nf4",  # 指定量化類型
        bnb_4bit_compute_dtype=torch.float16,  # 設定計算數據類型，以加快運算速度
        bnb_4bit_use_double_quant=True  # 啟用雙重量化，提高準確性但略增加記憶體用量
    )
    
    # 載入模型和分詞器
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,  # 使用新的量化配置
        torch_dtype=torch.float16,  # 設定模型的預設數據類型，以優化記憶體使用量
        device_map="auto"  # 自動分配模型到可用的設備上（CPU或GPU）
    )
    
    # 配置 LoRA
    lora_config = LoraConfig(
        r=8,  # LoRA的秩，控制LoRA矩陣的大小，影響參數數量和計算複雜度
        lora_alpha=16,  # LoRA放大因子，調整LoRA參數的學習率 (可以先設成 2倍 r)
        target_modules=[
            "q_proj",    # 查詢投影
            "k_proj",    # 鍵值投影
            "v_proj",    # 數值投影
            "o_proj",    # 輸出投影
            "gate_proj", # 門控投影
            "up_proj",   # 上投影
            "down_proj"  # 下投影
        ],  # 指定 LoRA 作用的模型部分
        lora_dropout=0.05,  # 設定 LoRA 層的 dropout 率，用於防止過擬合
        bias="none",  # 關閉偏差項
        task_type="CAUSAL_LM"  # 指定任務類型為因果語言模型
    )
    
    model = prepare_model_for_kbit_training(model)  # 準備模型以支持 kbit 訓練
    model = get_peft_model(model, lora_config)  # 使用 LoRA 配置加速模型訓練    

    # move model to available device
    available_device = torch.device(
        'cuda' if torch.cuda.is_available() else 'cpu')
    model.to(available_device)
    
    return model, tokenizer


In [10]:
def tokenize_function(examples, tokenizer):
    """將文本轉換為token"""
    return tokenizer(
        examples["text"],
        truncation=True, # 截斷文本以符合模型的最大長度
        max_length=self.chunk_size,  # 設定最大長度
        padding="max_length" # 填充文本以符合模型的最大長度
    )

In [11]:
# 載入數據
dataset = load_data(f'{self.paths["data"]}/{self.exp_config["data"]["file_path"]}')

# 分割訓練集和驗證集
train_test_split = dataset.train_test_split(test_size=0.1, seed=self.exp_config["model"]["seed"], shuffle=True)
train_dataset_raw = train_test_split['train']
valid_dataset_raw = train_test_split['test']

In [12]:
# 準備模型和分詞器
model, tokenizer = prepare_model_and_tokenizer(self.exp_config["model"]["name"])

In [13]:
# 處理數據集
train_dataset = train_dataset_raw.map(
    lambda x: tokenize_function(x, tokenizer), # 將文本轉換為token
    batched=True, # 一次處理多個樣本
    remove_columns=dataset.column_names # 移除原始文本列
)

# 處理數據集
valid_dataset = valid_dataset_raw.map(
    lambda x: tokenize_function(x, tokenizer), # 將文本轉換為token
    batched=True, # 一次處理多個樣本
    remove_columns=dataset.column_names # 移除原始文本列
)

Map:   0%|          | 0/2923 [00:00<?, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

In [14]:
# create model checkpoint folder
model_path = f"{self.paths['models']}/{self.exp_config['model']['name']}/{self.exp_config['model']['description']}"
helper_function.remove_directory(model_path) # remove old model checkpoint
os.makedirs(model_path, exist_ok=True) # create new model checkpoint

# create logging folder
logging_dir = f'{self.paths["reports"]}/{self.exp_config["model"]["name"]}/{self.exp_config["model"]["description"]}/logs'
helper_function.remove_directory(model_path) # remove old model checkpoint
os.makedirs(logging_dir, exist_ok=True) # create new model checkpoint

In [15]:
# 訓練參數配置
training_args = TrainingArguments(
    output_dir = model_path, 
    logging_dir = logging_dir,
    **self.exp_config['training_args']
    )

In [16]:
# 設定數據整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # 是否隨機遮蔽token，關閉以進行因果語言模型訓練，即預測下一個token
)

In [17]:
# 創建訓練器
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(self.exp_config['model']['early_stopping_patience'])]
    # compute_metrics=compute_metrics, # 添加計算指標的函數
    # loss_fn=custom_loss,
)

In [18]:
gpu_dict = helper_function.check_device(model, trainer.args)
self.logger.info(f"-- Model is using {'GPU: ' + str(gpu_dict['device_idx']) if gpu_dict['device_idx'] != -1 else 'CPU'}")
self.logger.info(f"-- training_args is using {gpu_dict['model_gpu_idx']}")

2025-01-01 04:17:55 INFO -- Model is using GPU: 0
2025-01-01 04:17:55 INFO -- training_args is using cuda:0


In [19]:
# 開始訓練
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
20,19.273100,2.374624
40,18.497100,2.256217
60,17.642600,2.140254
80,16.757400,2.062040
100,16.452400,2.010084
120,15.943100,1.972733
140,15.634200,1.946516
160,15.694500,1.927047
180,15.314100,1.911805
200,15.230200,1.899343


Could not locate the best model at /home/juliekuo/projects/story_structure/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/TinyLlama/checkpoint-720/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=920, training_loss=14.150202112612517, metrics={'train_runtime': 5734.3556, 'train_samples_per_second': 55.804, 'train_steps_per_second': 3.488, 'total_flos': 2.3580822994944e+16, 'train_loss': 14.150202112612517, 'epoch': 5.054719562243502})

In [20]:
# log model, tokenizer and config file to mlflow
self.log_to_mlflow(model, tokenizer, log_model_flag = False)
self.finish_mlflow()

🏃 View run TinyLlama at: http://localhost:5000/#/experiments/110165045259297196/runs/f97f609de5e042128283f73b9186a831
🧪 View experiment at: http://localhost:5000/#/experiments/110165045259297196


### Test model response

In [21]:
def load_original_model(model_name):
    """載入原始模型"""    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    return model, tokenizer

In [22]:
def load_finetuned_model(model_name, model_description, checkpoint):
    """載入微調後的模型"""
    adapter_path = f"{root}/models/{model_name}/{model_description}/{checkpoint}"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # 載入基礎模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # 載入 LoRA 權重
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model = model.merge_and_unload() # 合併 LoRA 權重並釋放記憶體
    
    return model, tokenizer

In [23]:
def generate_response(model, tokenizer, prompt, max_length=512):
    """生成回應"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [24]:
question = "你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。"

In [25]:
print("=== 原始模型回應 ===")
model, tokenizer = load_original_model(self.exp_config["model"]["name"])
original_response = generate_response(model, tokenizer, question)
print(original_response)

=== 原始模型回應 ===
你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。
- 這本小說兩個篇則是一起講的，故事主軸是希臘的神話傳說，但故事的兩個篇則是從不同的角度講的，一個是希臘的圖書傳說，另一個是希臘的神話傳說。
- 第一個篇講的是希臘的神話傳說，傳說對象是希臘的神，他們的存在是由當今的希臘對象所掌握的概念所創造，在他們的心中，所有的概念都是神的概念，但兩者之間的關係歷久不斷的變化，所有的神的概念，在希臘的神話傳說中，都沒有兩者相當的同質。
- 第二個篇則是希臘的圖書傳說，圖書傳說是希臘的神話傳說，它們的主軸是希臘的書，而不是希臘的神。希臘的圖書傳說的主軸是希臘的書，而不是希臘的神。

該小��


In [26]:
print("=== 微調後模型回應 ===")
model, tokenizer = load_finetuned_model(
    self.exp_config["model"]["name"], 
    self.exp_config["model"]["description"], 
    self.exp_config["model"]["checkpoint"]
    )
finetuned_response = generate_response(model, tokenizer, question)
print(finetuned_response)

=== 微調後模型回應 ===
你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。
卡里普索·傑克森是一位著名的諸神考察官。在他們的幾個演變中，他們雖然給我們很多問題，但雖然我們很抱怨，但卻也很快忘記了。
波西傑克森是一位臭糟糟的兒童。他的倒是一面漂亮。對於他，我們很熟悉了。他是我們最後一次發現的巨人。對於他，我們很熟悉。他的倒是一面漂亮。
我們在父親的大學裡，當時是在訓練學生。從一個劍傷痛的小女孩到一個倒霉的老實閃爍的傢伙，這是我們的演變。
我們終於找到了波西傑克森。他的眼睛漆黑了，但他的臉體卻十分穩固。我們找到了他的老朋友。
我們跟在他們身後的路上，對於他的這


In [27]:
question = '''
選項: 1. 離開 2. 說服 3. 戰鬥
選擇: 2. 說服

產生後續情節
'''                           

finetuned_response = generate_response(model, tokenizer, question)
print(finetuned_response)


選項: 1. 離開 2. 說服 3. 戰鬥
選擇: 2. 說服

產生後續情節
雖然貝理雅瑞斯對於離開說服的概念並沒有考慮，但是我們承諾說服他們。

30分鐘後

尋求者們坐在旁邊的船席上，幾乎都吵了。
盧克沒有辦法為它們讓一切過好。
「請幫忙找個安全的距離吧！」他問。
我們讓他們往後走。

安娜貝絲和格洛弗跟著我們。
「我們一定不能走著這個路啊！」安娜貝絲說。
「你們就是在尋求變換個人的帳號嗎？」
安娜貝絲發出一聲咒語。
格洛弗對我們說：「嘿，我們兩個都穿著同一套衣服，哼，沒關係。」
安娜貝絲說：「我們是尋求者嗎？」
「是的，」我說，
